# Preprocesamiento utilizando mediapipe

In [1]:
pip install mediapipe opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

In [5]:
# Configuración de rutas
input_base_path = r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/FaceForensics++_C23"
output_path = r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe"
deepfake_folders = ["DeepFakeDetection", "Deepfakes", "Face2Face", "FaceShifter", "FaceSwap"]
original_folder = "original"


In [11]:
# Parámetros
max_original_videos = 1000
max_fake_videos_per_folder = 200
target_frame_count = 16  # frames por video
output_img_size = (256, 256)

# Inicializar MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

def extract_landmark_vector(landmarks, image_shape):
    h, w, _ = image_shape
    coords = [(lm.x * w, lm.y * h) for lm in landmarks.landmark]

    def dist(p1, p2):
        return np.linalg.norm(np.array(coords[p1]) - np.array(coords[p2]))

    return np.array([
        dist(33, 263),  # entre ojos
        dist(61, 291),  # ancho de boca
        dist(1, 199),   # altura nariz
        dist(33, 61),   # ojo izq - boca izq
        dist(263, 291), # ojo der - boca der
    ])

def crop_face_from_landmarks(landmarks, image_bgr):
    h, w, _ = image_bgr.shape
    coords = [(int(lm.x * w), int(lm.y * h)) for lm in landmarks.landmark]
    xs, ys = zip(*coords)
    x_min, x_max = max(min(xs)-20, 0), min(max(xs)+20, w)
    y_min, y_max = max(min(ys)-20, 0), min(max(ys)+20, h)
    face_crop = image_bgr[y_min:y_max, x_min:x_max]
    return cv2.resize(face_crop, output_img_size)

def process_video(video_path, output_dir, label, num_frames_needed=16, candidate_frames=60):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < 1:
        cap.release()
        return

    frame_indices = np.linspace(0, total_frames - 1, candidate_frames).astype(int)
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    saved_images = []
    saved_vectors = []

    for idx, frame_idx in enumerate(frame_indices):
        if len(saved_images) >= num_frames_needed:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            continue

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            try:
                face_img = crop_face_from_landmarks(landmarks, frame)
                vector = extract_landmark_vector(landmarks, frame.shape)

                saved_images.append((idx, face_img))
                saved_vectors.append((idx, vector))
            except:
                continue

    cap.release()

    if len(saved_images) < num_frames_needed:
        print(f"[DESCARTADO] Solo se generaron {len(saved_images)}/16 muestras válidas para el video: {video_id}")
        return

    os.makedirs(output_dir, exist_ok=True)
    for i, (idx, img) in enumerate(saved_images[:num_frames_needed]):
        img_path = os.path.join(output_dir, f"{video_id}_{i}.jpg")
        cv2.imwrite(img_path, img)

    for i, (idx, vec) in enumerate(saved_vectors[:num_frames_needed]):
        vec_path = os.path.join(output_dir, f"{video_id}_{i}.npy")
        np.save(vec_path, vec)


def process_folder(folder_name, max_videos, label):
    folder_path = os.path.join(input_base_path, folder_name)
    output_dir = os.path.join(output_path, folder_name)
    os.makedirs(output_dir, exist_ok=True)

    videos = [f for f in os.listdir(folder_path) if f.endswith(".mp4")]
    for video in tqdm(videos[:max_videos], desc=f"Procesando {folder_name}"):
        video_path = os.path.join(folder_path, video)
        process_video(video_path, output_dir, label)

In [12]:
# Ejecutar preprocesamiento
process_folder(original_folder, max_original_videos, label="real")
for folder in deepfake_folders:
    process_folder(folder, max_fake_videos_per_folder, label="fake")

Procesando original:   3%|▎         | 34/1000 [00:14<13:03,  1.23it/s]

[DESCARTADO] Solo se generaron 13/16 muestras válidas para el video: 033


Procesando original:  10%|▉         | 97/1000 [00:48<36:14,  2.41s/it]

[DESCARTADO] Solo se generaron 0/16 muestras válidas para el video: 096


Procesando original:  16%|█▌        | 157/1000 [01:14<09:51,  1.42it/s]

...

Procesando FaceSwap:  48%|████▊     | 97/200 [00:46<04:03,  2.36s/it]

[DESCARTADO] Solo se generaron 0/16 muestras válidas para el video: 096_101


Procesando FaceSwap:  78%|███████▊  | 157/200 [01:09<00:27,  1.59it/s]

[DESCARTADO] Solo se generaron 3/16 muestras válidas para el video: 156_243


Procesando FaceSwap: 100%|██████████| 200/200 [01:28<00:00,  2.27it/s]
